# Investigate Wind Direction

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import math
import numpy as np
import cartopy.crs as ccrs
import geopandas as gpd
import glob
import moviepy.editor as mpy

from palettable.colorbrewer.diverging import RdBu_11

# Load Alpine-3D netCDF file

In [ ]:
# ds = xr.open_dataset("../output/grids/a3d_grids.nc")
ds = xr.open_dataset("/scratch/summit/erke2265/a3d_grids.nc")

# Plot wind speed and direction from Alpine-3D

In [ ]:
# SNOWPACK topography 
dem = np.flipud(np.loadtxt("../input/surface-grids/dem.asc", skiprows=6))

'''
Calculates U and V components of wind from wind speed and direction. 
Wind direction is the meteorological wind direction (i.e. direction the wind is coming from in degrees)
'''
def calc_U_V(speed, direction):
    U = -speed * np.sin(math.pi / 180 * direction)
    V = -speed * np.cos(math.pi / 180 * direction)
    return U, V

# Get U and V components
U, V = calc_U_V(ds['ws'], ds['dw'])
VW_max = ds['ws'].max()
VW_min = 0

# Calculate
dswe = 1000 * (ds['swe'] - ds['swe'].isel(time=0))

# Get lat/lon
x_snowpack = ds['easting']
y_snowpack = ds['northing']


# Make movie
# Clear old images, gifs,and movies
!mkdir -p movie_frame
!rm -f movie.gif
!rm -f movie.mp4
!rm -f movie_frame/*

for time_step in range(0, len(ds['time']), 1):
# for time_step in range(90, 110):
    print(time_step)
    plt.clf()
    
    # Plot map of mean wind
    plt.figure(figsize=(30,15))

    # DEM
    contour_levels = np.linspace(dem.min(), dem.max(), 25)
    contour = plt.contour(x_snowpack.values, y_snowpack.values, dem, contour_levels, linestyles='solid', colors='white')
    plt.clabel(contour, fmt = '%.0f', inline = True)

    # Delta SWE
    colormap = RdBu_11.mpl_colormap # works
    plt.pcolor(x_snowpack.values, y_snowpack.values, dswe.isel(time=time_step), \
               cmap = colormap, vmin=-np.abs(dswe).max(), vmax=np.abs(dswe).max())
    plt.colorbar()

    # Wind speed and direction (still need to normalize arrows so that length means the same thing for all time steps)
    spacing = 6
    northing_ind = np.arange(0, len(ds['northing']), spacing)
    easting_ind = np.arange(0, len(ds['easting']), spacing)
    plt.quiver(x_snowpack[easting_ind].values, y_snowpack[northing_ind].values, \
               U.isel(time=time_step)[northing_ind, easting_ind], V.isel(time=time_step)[northing_ind, easting_ind], scale=250)
    plt.title(ds['time'].isel(time=time_step).values, fontsize=20)
    
    #Save Figure with image number zero padding 
    if time_step < 10: # 1 digit
        plt.savefig("movie_frame/frame_000" + str(time_step) + ".png", dpi=100)
    elif time_step < 100 and time_step > 9: # 2 digits
        plt.savefig("movie_frame/frame_00" + str(time_step) + ".png", dpi=100)
    elif time_step < 1000 and time_step > 99: # 3 digits
        plt.savefig("movie_frame/frame_0" + str(time_step) + ".png", dpi=100)
    else: # 4 digits
        plt.savefig("movie_frame/frame_" + str(time_step) + ".png", dpi=100)
    plt.close()
    
# Make a .mp4 movie and gif
file_list = sorted(glob.glob('movie_frame/*.png'))
clip = mpy.ImageSequenceClip(file_list, fps=30)
clip.write_videofile('movie.mp4')
# clip.write_gif('movie.gif')
    

# # Plot map of wind direction
# plt.figure(figsize=(20,10))
# plt.pcolor(x_snowpack.values, y_snowpack.values, ds['dw'].isel(time=time_step), cmap='twilight')
# plt.colorbar()

# # Plot horizontal transect of wind direction
# plt.figure(figsize=(10,5))
# ds['dw'][0,50,:].plot()


# Plot a transect of wind speed and topography

In [ ]:
# Calculate transects
ind = 25
ws_transect = ds['ws'].isel(time=0)[:, ind]
# dem_transect = dem[:, ind]
dem_transect = np.flip(np.gradient(dem[:, ind]))

# Plot both transects
fig, ax1 = plt.subplots(figsize=(15,5))

# Wind Speed
color = 'tab:red'
ax1.set_ylabel('Wind Speed', color=color)  
ax1.plot(ws_transect, color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax2 = ax1.twinx() 

# Elevation
color = 'tab:blue'
ax2.set_ylabel('Elevation Gradient', color=color)  
ax2.plot(dem_transect, color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Settings
fig.tight_layout()  
plt.title("Vertical Transect at Easting = " + str(float(ws_transect['easting'].values)))
plt.show()


# Plot MERRA-2 winds

In [ ]:
# Define upper left and lower right A3D domain corners in epsg 3031
ncol = 168
nrow = 107
left = -1542000 # done
bottom = -109000 # done
top = bottom + 1000 * (nrow -1)
right = left + 1000 * (ncol -1)

# Look at MERRA-2
M2_U_path = "/scratch/summit/erke2265/MERRA2/U10M_hourly_1980.nc"
M2_V_path = "/scratch/summit/erke2265/MERRA2/V10M_hourly_1980.nc"

# Mean U and V components
M2_U = xr.open_dataset(M2_U_path)
M2_U = M2_U['U10M']
M2_U_mean = M2_U[0,:,:].values
# M2_U_mean = M2_U.mean(dim='time').values

M2_V = xr.open_dataset(M2_V_path)
M2_V = M2_V['V10M']
M2_V_mean = M2_V[0,:,:].values
# M2_V_mean = M2_V.mean(dim='time').values

# Calculate mean wind speed.
ws_mean = np.sqrt( np.power(M2_U_mean,2) + np.power(M2_V_mean,2))

# Load coastlines
df = gpd.read_file("/pl/active/nasa_smb/Data/ADD_Coastline_low_res_polygon.shp")
crs_epsg = ccrs.SouthPolarStereo()
df_epsg = df.to_crs(epsg='3031')

# Generate figure 
fig, axs = plt.subplots(1, 1, subplot_kw={'projection': crs_epsg},
                        figsize=(15, 15))

# Plot coastlines
# axs.set_extent((-180, 180, -90, -60), ccrs.PlateCarree())
axs.set_extent((-110, -70, -85, -70), ccrs.PlateCarree())
axs.add_geometries(df_epsg['geometry'], crs=crs_epsg,
                      facecolor='none', edgecolor='black')

# Plot winds
vector_crs = ccrs.PlateCarree()
space = 1
ws = plt.pcolor(M2_U['lon'].values, M2_U['lat'].values, ws_mean, transform=vector_crs, alpha=0.5)
cbar = plt.colorbar(ws)
streamlines = axs.streamplot(M2_U['lon'].values, M2_U['lat'].values, M2_U_mean, \
            M2_V_mean, color='k', transform=vector_crs, density=5)

# Plot model domain
plt.scatter(left, bottom, c='r', label="A3D Domain")
plt.scatter(left, top, c='r')
plt.scatter(right, bottom, c='r')
plt.scatter(right, top, c='r')
plt.savefig("MERRA-2.pdf", format='pdf', dpi=100)